In [15]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

target_size = (224, 224)
batch_size = 64

train_generator = datagen.flow_from_directory(
    "train",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    "train",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='rgb',
    subset='validation',
    shuffle=False
)

classes = list(train_generator.class_indices)
print(classes)

In [ ]:
base_model = load_model('first_model.keras')

for layer in base_model.layers:
    layer.trainable = False

input_shape = base_model.input_shape[1:]

inputs = Input(shape=input_shape)
x = base_model(inputs)
x = Dense(128, activation='relu')(x)

predictions = Dense(120, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions, name="Dog_Breed")
model.build(input_shape)
model.summary()

In [ ]:
print(f"Model input shape: {model.input_shape}")

print(f"Train generator batch size: {train_generator.batch_size}")
print(f"Train generator image shape: {train_generator.image_shape}")


In [19]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
early_stopping = EarlyStopping(
    patience=15,
    monitor="val_accuracy",
    restore_best_weights=True
)
model_checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max"
)
reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy",
    patience=5,
    min_delta=0.00005,
    min_lr=0.00005,
    restore_best_weights=True
)

callbacks = [early_stopping, model_checkpoint, reduce_lr]

In [ ]:
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=callbacks)